I’ll guide you step-by-step to **set up the IAM role** used by AWS Glue, **create a policy**, and then **attach the policy** to the IAM role using the AWS CLI.

### **Step 1: Create an IAM Role for AWS Glue**

The role used by AWS Glue will need the necessary permissions for Glue operations. We'll create a new role for AWS Glue and assign it the necessary permissions.

#### A. **Create a Trust Policy Document for Glue Role**

Glue needs a role that allows it to be assumed by AWS Glue. You will create a trust policy that allows AWS Glue to assume the role.

Create a file named `trust-policy.json`:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "glue.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```

This trust policy allows AWS Glue to assume the role.

#### B. **Create the IAM Role**

Now, create the role using the trust policy:

```bash
    aws iam create-role --role-name AWSGlueServiceRole --assume-role-policy-document trust-file://policy.json
```

This command will return information about the new role, including its ARN.

---

### **Step 2: Create a Policy for AWS Glue Permissions**

Next, you need to create a policy that grants the necessary permissions (such as access to Glue, S3, and other services).

#### A. **Create the Permissions Policy Document**

Create a file named `GlueFullAccessPolicy.json` to specify the permissions:

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "glue:*",
        "s3:*",
        "rds:*",
        "logs:*"
      ],
      "Resource": "*"
    }
  ]
}
```

#### B. **Create the IAM Policy**

Use the following command to create the policy using the `GlueFullAccessPolicy.json` file:

```bash
aws iam create-policy \
  --policy-name GlueFullAccessPolicy \
  --policy-document file://GlueFullAccessPolicy.json
```

After running this command, you will get the ARN of the policy you just created.

---

### **Step 3: Attach the Policy to the Role**

Now that you have created the IAM role and policy, you need to attach the policy to the role so that AWS Glue can use it.

#### A. **Attach the Policy to the Role**

Use the following command to attach the policy to the role:

```bash
aws iam attach-role-policy --role-name AWSGlueServiceRole --policy-arn arn:aws:iam::<account-id>:policy/GlueFullAccessPolicy
```

Replace `<account-id>` with your AWS account ID, which you can find in the **AWS Management Console** or using the command:

```bash
aws sts get-caller-identity --query Account --output text
```

If you used the AWS-managed policy `AWSGlueServiceRole`, you can attach it directly:

```bash
aws iam attach-role-policy --role-name AWSGlueServiceRole --policy-arn arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole
```

---

### **Step 4: Verify the Role and Policy Attachment**

To verify that the policy was successfully attached to the role, you can run:

```bash
aws iam list-attached-role-policies --role-name AWSGlueServiceRole
```

This will list all the policies attached to the `AWSGlueServiceRole`.

---

### **Step 5: Create an S3 Bucket (optional)**

If you want to store your Glue job outputs in S3, you will need to create an S3 bucket. Here's the CLI command to create an S3 bucket:

```bash
aws s3 mb s3://my-glue-bucket
```

Replace `my-glue-bucket` with your preferred S3 bucket name.

---

### **Step 6: Check the Role's Trust Policy**

To ensure the trust policy was created correctly for the `AWSGlueServiceRole`, you can run:

```bash
aws iam get-role --role-name AWSGlueServiceRole --query "Role.AssumeRolePolicyDocument"
```

This should return the trust policy allowing AWS Glue to assume the role.

---

### **Conclusion**

Now you have created the **IAM role** for AWS Glue (`AWSGlueServiceRole`), assigned the **necessary permissions** to that role (via `GlueFullAccessPolicy`), and attached the policy to the role. Your role is now ready to be used by AWS Glue to access resources like Glue, S3, RDS, and more.

You can use this role with AWS Glue crawlers and jobs to connect to your databases (e.g., MariaDB, MongoDB) or any other sources.